# View html output of this notebook here:
https://htmlpreview.github.io/?https://github.com/zagoodman/swallow/blob/main/jupyter/assemble_data.html

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Prep" data-toc-modified-id="Prep-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Prep</a></span></li><li><span><a href="#Processed-strain-data" data-toc-modified-id="Processed-strain-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Processed strain data</a></span><ul class="toc-item"><li><span><a href="#Exploratory-functions" data-toc-modified-id="Exploratory-functions-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Exploratory functions</a></span></li><li><span><a href="#Data-cleaning-function" data-toc-modified-id="Data-cleaning-function-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Data cleaning function</a></span></li><li><span><a href="#Plotting-functions" data-toc-modified-id="Plotting-functions-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Plotting functions</a></span></li><li><span><a href="#Get-dataframe-containing-data-for-all-study-participants" data-toc-modified-id="Get-dataframe-containing-data-for-all-study-participants-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Get dataframe containing data for all study participants</a></span></li></ul></li></ul></div>

This file takes 'processed' strain data and returns a dataframe lined-up on when the swallow begins. Currently, the process is optimized for one user's data, though with slight tweaks and more consistent data recording, the process should work similarly for future data records.

## Prep

For this script to work, your folders should be as follows:
- base directory
    - data
        - processed_strain
            - andrew
                - andrew_5_strain_swallow.mat
                - andrew_10_strain_swallow.mat
                - etc
            - etc.
    - jupyter
        - this file

In [ ]:
# import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
import matplotlib
from matplotlib import pyplot as plt
from scipy.io import loadmat

In [ ]:
# custom plot settings

size=20
params = {'legend.fontsize': 'large',
          'legend.title_fontsize': size*0.75,
          'legend.frameon': False,
          'figure.figsize': (10,6),
          'axes.labelsize': size,
          'axes.titlesize': size,
          'xtick.labelsize': size*0.75,
          'ytick.labelsize': size*0.75,
          'axes.titlepad': 10,
          'figure.subplot.hspace': 0.9,
          'font.sans-serif': 'Arial'}
# matplotlib.rcParams.keys() # to see editable options
plt.rcParams.update(params)

## Processed strain data

### Exploratory functions
These help understand the raw data and check that they are as expected.

In [ ]:
def plot_raw(name='B', ml=5):
    """
    Plots the raw time series for a given person and ml
    """
    mat = loadmat('../data/processed_strain/'+name+'/'+name+'_' + str(ml) + '_strain_swallow.mat')
    for i in range(0, 10): 
        time = mat['water_'+str(ml)+'ml'][0][i][0][0][0][0]
        strain = mat['water_'+str(ml)+'ml'][0][i][0][0][1][0]
        
        # keep only first swallow data
        time = time[:250]
        strain = strain[:250]
        
        # normalize strain
        strain = strain - strain[0]
        
        # normalize time
        time = time - time[0]
        
        # get df
        dftmp = pd.DataFrame({'time':time, 'strain':strain})
        dftmp = dftmp.loc[dftmp.time.between(0, 30)]
        
        # plot
        plt.plot(dftmp.time, dftmp.strain, alpha=0.3, c='blue')
        
    plt.show()
        
        
plot_raw('B', 5)

In [ ]:
def plot_smoothed(name='B', ml=5):
    """
    Plots, for a given ml, a moving average of the strain time series.
    The raw is in blue, moving average in red.
    """
    mat = loadmat('../data/processed_strain/'+name+'/'+name+'_' + str(ml) + '_strain_swallow.mat')
    for i in range(0, 10): 
        time = mat['water_'+str(ml)+'ml'][0][i][0][0][0][0]
        strain = mat['water_'+str(ml)+'ml'][0][i][0][0][1][0]
        
        # keep only first swallow data
        time = time[:250]
        strain = strain[:250]
        
        # normalize strain
        strain = strain - strain[0]
        
        # normalize time
        time = time - time[0]
        
        # moving average
        w = 8 # window
        # add zeros to end of vec
#         cumsum_vec = np.insert(strain, len(strain), np.zeros(int(np.floor(w/2))))
        # add zeros to front of vec
        cumsum_vec = np.insert(strain, 0, np.zeros(int(np.ceil(w/1))))
        # calc cumulative sum, then moving average
        cumsum_vec = np.cumsum(cumsum_vec)
        ma_vec = (cumsum_vec[w:] - cumsum_vec[:-w]) / w
        
        # get df
        dftmp = pd.DataFrame({'time':time, 'strain':strain, 'strain_sm':ma_vec})
        dftmp = dftmp.loc[dftmp.time.between(6, 13)]
        
        # plot
        plt.plot(dftmp.time, dftmp.strain, alpha=0.1, c='blue')
        plt.plot(dftmp.time, dftmp.strain_sm, alpha=0.3, c='red')
        
    plt.show()
        
        
plot_smoothed('B', 5)

### Data cleaning function
These help turn the raw data into something usable. ***get_person_df*** identifies the swallow in the raw strain data, then normalizes the swallow to start at time=0 and the strain to be zero when the swallow starts.

In [ ]:
def get_person_df(name='B'):
    """
    Fetches data for person 'name', returns df with 
    normalized strain data.
    
    Different: this function uses slope to find the swallow start
    """

    # init df and colors
    df = pd.DataFrame()
    colors = ['red', 'orange', 'indigo', 'green', 'blue', 'violet']

    # loop over 6 swallow amounts
    for j in [5, 10, 15, 20, 25, 30]:
        mat = loadmat('../data/processed_strain/' + name + '/' + name + \
                      '_' + str(j) + '_strain_swallow.mat')

        # loop over 10 attempts
        for i in range(10):

            # get time and strain
            time = mat['water_'+str(j)+'ml'][0][i][0][0][0][0]
            strain = mat['water_'+str(j)+'ml'][0][i][0][0][1][0]

            # keep only first swallow data
            time = time[:250]
            strain = strain[:250]

            # normalize strain
            # apply moving average to strain
            w = 6 # window size
            cumsum_vec = np.insert(strain, 0, [strain[0]] * w)
            cumsum_vec = np.cumsum(cumsum_vec)
            ma_vec = (cumsum_vec[w:] - cumsum_vec[:-w]) / w
            
            # get slopes
            slope_vec = np.insert((ma_vec[1:] - ma_vec[:-1])/0.630, 0, 0)
            # plot to check work
#             print(max(slope_vec))
#             fig, ax = plt.subplots(3, 1)
#             ax[0].plot(time, strain)
#             ax[1].plot(time, ma_vec)
#             ax[2].plot(time, slope_vec)

            # calculate critical slope, a weighted avg between 
            #  standard deviation of noise and the max slope
            # TODO - better optimize this?
            stddev = slope_vec[:150].std() # std deviation of noise
            maxslope = max(slope_vec) # max slope
            critical_slope = min((4*max(stddev * 3, 0.005) + 1*maxslope)/5, maxslope)
            
            # find where slope_vec is > critical slope
            slope_idx = min([i for i, x in enumerate(slope_vec >= critical_slope) if x])
            
            # now find the next prev idx where slope is negative AND 
            #  next two slopes are flat or negative (< mcrit)
            # valley_idx is the index of the valley before the swallow
            valley_idx = 0
            mcrit = 0.005
            for k in range(slope_idx-1, 0, -1):
                if slope_vec[k] < 0 and slope_vec[k-1] < mcrit and slope_vec[k-2] < mcrit:
                    valley_idx = k # TODO k? or k-1, k+1, etc?
                    break

            # normalize strain to zero at valley
            strain = strain - strain[valley_idx]
            ma_vec = ma_vec - ma_vec[valley_idx]

            # normalize time to zero at valley
            time = time - time[valley_idx]

            # construct temporary df with new data
            dftmp = pd.DataFrame({'time':time, 'strain':strain, 'strain_sm':ma_vec, 'slope':slope_vec})
            dftmp['s_vol'] = j
            dftmp['test'] = int(i + (np.floor(j / 5) - 1) * 10)
            dftmp['slope_idx'] = slope_idx
            dftmp['valley_idx'] = valley_idx

            # add to complete df
            df = pd.concat([df, dftmp]).\
                reset_index(drop=True)

            plt.plot(dftmp.time, dftmp.strain, label=j if i == 0 else "", \
                     alpha=0.2, c=colors[int((j-1)/5)])
    return df

df = get_person_df()
plt.legend()
display(df.head())

In [ ]:
# Let's check the swallow identification.

print(df.loc[df.time < -10, 'test'].value_counts().head(3))
# print(df.loc[df.time > 7.5, 'test'].value_counts().head(3))

# 43 and 27 have different challenges
# 43 lacks a large first peak - the second peak dominates
# 27 has a positive strain leading into the swallow
for t in [43]:
    dfsub = df.loc[df.test == t]
    fig, ax = plt.subplots(3, 1, figsize=(8,6))
    ax[0].plot(dfsub.time, dfsub.strain)
    ax[0].title.set_text('raw strain')
    ax[1].plot(dfsub.time, dfsub.strain_sm)
    ax[1].title.set_text('smoothed strain')
    ax[2].plot(dfsub.time, dfsub.slope)
    ax[2].title.set_text('slope of smoothed strain')
    plt.show()

### Plotting functions
***plot_strain_ts*** plots the time series for strain. Supply a dataframe containing the strain data, then set options:
1. ```raw```: include the raw data in the background of the plot
2. ```smooth```: use local average smoothing in plot

In [ ]:
# plot averages

def plot_strain_ts(df, name="", timestart=-1, timeend=3, vols=[5,10,15,20], raw=True, smooth=False, colors=colors):
    """
    Plots the time series of the average strain for each swallow volume in 'vols'.
    Set raw=True to depict the raw data in the background of the plot.
    Set smooth=True for local averages of the time series.
    """
    # if smooth, keep only moving average
    dff = df.copy()
    if smooth:
        dff.drop('strain', 1, inplace=True)
        dff.rename(columns={'strain_sm': 'strain'}, inplace=True)
        
    # aggregate df
    dff = dff.loc[dff.time.between(timestart, timeend)]
    dffagg = dff.copy()
    dffagg.time = dffagg.time.apply(lambda x: round(x, 1))
    dffagg = dffagg.groupby(['time', 's_vol'])['strain'].agg('mean').reset_index()

    # plot    
    fig, ax = plt.subplots(1, 1, figsize=(12,8))
    for v in vols:
        dfsub = dffagg.loc[dffagg.s_vol == v]
        ax.plot(dfsub.time, dfsub.strain, color=colors[int(v/5) - 1], label=v)
        if raw:
            for t in range(0, 10):
                dfsub2 = dff.loc[(dff.test == int((v/5 - 1) * 10 + t)) & (dff.s_vol == v)]
                ax.plot(dfsub2.time, dfsub2.strain, alpha=0.09, color=colors[int(v/5) - 1], label="")

    ax.legend(title='Volume (ml)')
    plt.xlabel('Time Since Swallow Start (s)')
    plt.ylabel('Normalized Strain')
    if name != "":
        strappend = ' for {}'.format(name)
    else:
        strappend = ''
    if smooth:
        plt.title('Average Strain{}, Smoothed'.format(strappend))
    else:
        plt.title('Average Strain{}, Unsmoothed'.format(strappend))
    plt.show()

# select colors
# colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown']
colors = matplotlib.cm.Reds(np.linspace(0, 1, 8))[2:]

vols=[5, 10, 15, 20, 25, 30]
plot_strain_ts(df, vols=vols, name='Person 1', colors=colors, raw=False)
plot_strain_ts(df, vols=vols, name='Person 1', smooth=True, colors=colors, raw=False)

In [ ]:
# select colors
colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown']
# colors = matplotlib.cm.Reds(np.linspace(0, 1, 8))[2:]

vols=[5, 10, 15, 20, 25, 30]
plot_strain_ts(df, vols=vols, name='Person 1', colors=colors)
plot_strain_ts(df, vols=vols, name='Person 1', smooth=True, colors=colors)

In [ ]:
colors = matplotlib.cm.viridis_r(np.linspace(0, 1, 8))[2:]

vols=[5, 10, 15, 20, 25, 30]
plot_strain_ts(df, vols=vols, name='Person 1', colors=colors)
plot_strain_ts(df, vols=vols, name='Person 1', smooth=True, colors=colors)

### Get dataframe containing data for all study participants
Here we concatenate the data from all four study participants for whom we have data.

In [ ]:
# get df for all people

dfall = pd.DataFrame()

for n in ['B', 'andrew', 'rachel', 'steve']:
    print(n)
    dftmp = get_person_df(n)
    dftmp['name'] = n
    dfall = pd.concat([dfall, dftmp])
    
dfall

In [ ]:
# check name's data

name = 'andrew'
vols = [5, 10]

for v in vols:
    plot_raw(name=name, ml=v)
    plot_smoothed(name=name, ml=v)

dfsub = dfall.loc[dfall.name == name]
plot_strain_ts(dfsub, name=name, vols=vols, smooth=False)
plot_strain_ts(dfsub, name=name, vols=vols, smooth=True)


In [ ]:
# check name's data

name = 'steve'
vols = [5, 10]

for v in vols:
    plot_raw(name=name, ml=v)
    plot_smoothed(name=name, ml=v)

dfsub = dfall.loc[dfall.name == name]
plot_strain_ts(dfsub, name=name, vols=vols, smooth=False)
plot_strain_ts(dfsub, name=name, vols=vols, smooth=True)


In [ ]:
# aggregate within time/volume, plot averages

dfagg = dfall.copy()
dfagg.time = dfagg.time.apply(lambda x: round(x, 1))
dfagg = dfagg.groupby(['time', 's_vol', 'name'])['strain'].agg('mean').reset_index()
dfagg = dfagg.loc[dfagg.time.between(-5, 5)]
names = ['B', 'andrew', 'rachel', 'steve']

f, ax = plt.subplots(4, 1, sharex=True, figsize=(10,8))


for k in range(len(names)):
    n = names[k]
    for l in range(1, 3):
        ml = 5 * l
        dfsub = dfagg.loc[(dfagg.name == n) & (dfagg.s_vol == ml), :]
        ax[k].plot(dfsub.time, dfsub.strain,
                 label = ml)
        ax[k].set_title(n)
        ax[k].legend()
    
plt.show()